<a href="https://colab.research.google.com/github/cleysonl/Pytorch/blob/master/DeepNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>